In [2]:
import os
from dotenv import load_dotenv

load_dotenv()  # .env 파일 읽어오기

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
PINECONE_API_KEY=os.environ['PINECONE_API_KEY']

In [ ]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY) #pc객체를 이용하여 인덱스를 만들고, 삭제하고, 데이터를 다룰 수 있음

In [4]:
index_name = "wiki"
for idx in pc.list_indexes():
    # 인덱스 이름이 "wiki"와 일치하는 경우 해당 인덱스를 삭제합니다.
    if idx.name == index_name:
        pc.delete_index(idx.name)

In [5]:
from pinecone import ServerlessSpec
pc.create_index(
    name=index_name,
    dimension=1536,  # 모델 차원
    metric="cosine",  # 모델 메트릭
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

{
    "name": "wiki",
    "metric": "cosine",
    "host": "wiki-6aohn7n.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 1536,
    "deletion_protection": "disabled",
    "tags": null
}

In [6]:
index = pc.Index(index_name)  #'wiki' 인덱스 가져오기

index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}

In [8]:
%pip install -qU datasets

Note: you may need to restart the kernel to use updated packages.


In [10]:
from datasets import load_dataset
data = load_dataset("wikimedia/wikipedia", "20231101.ko", split='train[:100]') 

c:\Users\Admin\miniconda3\envs\prompting_env\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\datasets--wikimedia--wikipedia. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regul

In [12]:
data

Dataset({
    features: ['id', 'url', 'title', 'text'],
    num_rows: 100
})

In [11]:
data[0]

{'id': '5',
 'url': 'https://ko.wikipedia.org/wiki/%EC%A7%80%EB%AF%B8%20%EC%B9%B4%ED%84%B0',
 'title': '지미 카터',
 'text': '제임스 얼 카터 주니어(, 1924년 10월 1일~)는 민주당 출신 미국의 제39대 대통령(1977년~1981년)이다.\n\n생애\n\n어린 시절 \n지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.\n\n조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.\n\n정계 입문 \n1962년 조지아주 상원 의원 선거에서 낙선하였으나, 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주지사 선거에 낙선하지만, 1970년 조지아 주지사 선거에서 당선됐다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.\n\n대통령 재임 \n\n1976년 미합중국 제39대 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책으로 내세워서, 많은 지지를 받았는데 제럴드 포드 대통령을 누르고 당선되었다.\n\n카터 대통령은 에너지 개발을 촉구했으나 공화당의 반대로 무산되었다.\n\n외교 정책 \n카터는 이집트와 이스라엘을 조정하여 캠프 데이비드에서 안와르 사다트 대통령과 메나헴 베긴 수상과 함께 중동 평화를 위한 캠프데이비드 협정을 체결했다. 이것은 공화당과 미국의 유대인 단체의 반발을 일으켰다. 그러나 1979년, 양국 간의 평화조약이 백악관에서 이루어졌다.\n\n소련과 제2차 전략 무기 제한 협상(SALT II)에 조인했다.\n\n카터는 1970년대 후반 당시 대한민국 등 인권 후진국의 국민들의 인권을 지키기 위해 노력했으

In [13]:
len(data)

100

In [14]:
for rec in data:
    if len(rec['text'])>35000: ##크기제한
        print('**')   

**
**


In [15]:
from langchain_openai import OpenAIEmbeddings
embedding=OpenAIEmbeddings(model='text-embedding-3-small') #모델 불러오기
embedding

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x0000023003EBD310>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x0000023004365A90>, model='text-embedding-3-small', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [16]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter=RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=20,
    length_function=len, \
    separators=["\n\n", "\n", " ", ""] #자르는 순서문자
)
splitter

In [20]:
import json
def calculate_metadata_size(metadata):
  return len(json.dumps(metadata, ensure_ascii=False).encode('utf-8'))

In [ ]:
from uuid import uuid4
import time
texts = []
metas = []
count = 0
batch_size = 20 #20개 청크씩 모아서 임베딩 후 업서트할 계획

In [ ]:
for i, sample in enumerate(data):
  full_text = sample['text'] #전문
  
  #메타데이터 구성
  metadata = {
    "wiki_id": str(sample['id']),
    "url": sample['url'],
    "title": sample['title'] #[:100]
  }
  
  # 스플릿터로 자르기
  chunks = splitter.split_text(full_text)
  #print(len(chunks))
  
  for i, chunk in enumerate(chunks):
    record = {
      'chunk_id':i,
      'full_text':full_text,
      **metadata
    }
    metadata_size = calculate_metadata_size(record)
    if metadata_size > 35000:
      continue
    
    texts.append(chunk)
    metas.append(record)
    count += 1
    
    # 청크 배치의 임베딩 -> 적재
    if count % batch_size == 0:
      ids = [str(uuid4()) for _ in range(len(texts))] #uuid4()로 unique ID 생성
      embeddings = embedding.embed_documents(texts) #20개 청크의 임베딩 수행
      index.upsert(
        vectors=zip(ids, embeddings, metas),
        namespace='wiki-ns1'
      )
      texts=[]
      metas=[]
      time.sleep(1)

## 질의 임베딩

In [24]:
question=["벨기에는 어디에 있나요?"]
emb_question=embedding.embed_documents(question)
emb_question

[[0.0029317522421479225,
  0.038531601428985596,
  0.0835522934794426,
  0.01172700896859169,
  -0.01034860871732235,
  -0.015395675785839558,
  0.011822436936199665,
  0.015851609408855438,
  -0.028034551069140434,
  -0.02203320525586605,
  -0.019255198538303375,
  0.05802006646990776,
  0.04222147539258003,
  -0.030833764001727104,
  0.0014698522863909602,
  0.0055931261740624905,
  -0.06930174678564072,
  0.01671045832335949,
  0.026719767600297928,
  0.02192717418074608,
  -0.023347988724708557,
  -0.022966276854276657,
  -0.032254576683044434,
  0.048646941781044006,
  0.042560774832963943,
  0.007125271484255791,
  0.01567135564982891,
  0.00029837735928595066,
  0.00684959115460515,
  -0.036156512796878815,
  0.022669389843940735,
  -0.0325726680457592,
  0.021344006061553955,
  -0.05038584768772125,
  0.014197527430951595,
  0.009839660488069057,
  0.045126721262931824,
  -0.0440664105117321,
  0.017866194248199463,
  -0.023560049012303352,
  -0.006059661507606506,
  0.00516370

In [26]:
# 파인콘 질의 실행
q_result=index.query(
    namespace="wiki-ns1",
    vector=emb_question,
    top_k=5,
    include_vector=False,
    include_metadata=True
)
q_result

{'matches': [{'id': '33005e17-427e-42e6-879b-6629fefaa279',
              'metadata': {'chunk_id': 11.0,
                           'full_text': '엔트로피(, )는 열역학적 계의 유용하지 않은 (일로 변환할 수 '
                                        '없는) 에너지의 흐름을 설명할 때 이용되는 상태 함수다. '
                                        '통계역학적으로, 주어진 거시적 상태에 대응하는 미시적 상태의 수의 '
                                        '로그로 생각할 수 있다. 엔트로피는 일반적으로 보존되지 않고, '
                                        '열역학 제2법칙에 따라 시간에 따라 증가한다. 독일의 물리학자 '
                                        '루돌프 클라우지우스가 1850년대 초에 도입하였다. 대개 기호로 '
                                        '라틴 대문자 S를 쓴다.\n'
                                        '\n'
                                        '정의 \n'
                                        '엔트로피에는 열역학적 정의와 통계학적인 정의, 두 가지의 관련된 '
                                        '정의가 있다. 역사적으로, 고전 열역학적 정의가 먼저 발전하였다. '
                                        '고전 열역학적인 관점에서, 그 이론은 원자나 분자 같은 수많은 '
                                      

In [27]:
result_ids=[r.id for r in q_result.matches]
result_ids

['33005e17-427e-42e6-879b-6629fefaa279',
 'cd9381cf-70c5-493e-9c9b-faa95f0a1247',
 '48e17787-8403-4a74-b9a1-78a5565dda1a',
 'a302e94e-23c5-4a5b-b443-aecbb9646955',
 'aebf25f5-4953-40ae-a027-818ca5754c78']

In [28]:
for r in q_result.matches:
    print(r.score, r.metadata)

0.306592971 {'chunk_id': 11.0, 'full_text': '엔트로피(, )는 열역학적 계의 유용하지 않은 (일로 변환할 수 없는) 에너지의 흐름을 설명할 때 이용되는 상태 함수다. 통계역학적으로, 주어진 거시적 상태에 대응하는 미시적 상태의 수의 로그로 생각할 수 있다. 엔트로피는 일반적으로 보존되지 않고, 열역학 제2법칙에 따라 시간에 따라 증가한다. 독일의 물리학자 루돌프 클라우지우스가 1850년대 초에 도입하였다. 대개 기호로 라틴 대문자 S를 쓴다.\n\n정의 \n엔트로피에는 열역학적 정의와 통계학적인 정의, 두 가지의 관련된 정의가 있다. 역사적으로, 고전 열역학적 정의가 먼저 발전하였다. 고전 열역학적인 관점에서, 그 이론은 원자나 분자 같은 수많은 성분들로 이루어져 있고, 학설의 안정성은 그러한 성분들의 평균적인 열특성으로 설명된다.\n이론을 구성하는 성분의 세부적인 성분들은 직접적으로 보이지 않는다. 그러나 그 특성은 온도, 압력, 엔트로피, 열용량과 같은 눈으로 볼 수 있는 평균적인 지표들에 의해 설명된다. 그 이론의 특성에 관한 고전적인 정의는 평형 상태임을 가정하였다. 엔트로피의 고전 열역학적 정의는 최근 비평형 열역학의 영역으로까지 확대되었다. 이후 엔트로피를 포함한 열특성은 눈으로 볼 수 없는 미시세계에서의 움직임에 대한 정역학적인 관점에서 새롭게 정의되었다. 그 예로 처음에는 기체로 여겨졌지만 시간이 지난 후에 광자, 음자, 스핀과 같은 양자 역학적으로 생각되는 뉴턴 입자가 있다. 이 이론의 특성에 대한 통계학적 설명은 고전적인 열역학을 사용하여 이론의 특성을 정의하는 것이 몇몇 변수의 영향을 받는 이론의 최종적인 상태를 예측하는데 있어서 점점 신뢰할 수 없는 예측 기술이 되어감으로 인하여 필수적인 것이 되었다.\n\n열역학적 정의 \n고전적 열역학에서는 엔트로피 S의 절대적 값은 정의할 수 없고, 대신 그 상대적 변화만 정의한다. 열적 평형을 이뤄 온도가 인 계에 열 를 가하였다고 하자. 이 경우 엔트로피의 증가

### Langchain의 api로 결과 비교하기

In [29]:
question

['벨기에는 어디에 있나요?']

In [33]:
from langchain_pinecone import PineconeVectorStore
vector_store=PineconeVectorStore(index=index, embedding=embedding, text_key='full_text')
docs=vector_store.similarity_search(query=question[0], k=5, namespace='wiki-ns1')
docs

[Document(id='33005e17-427e-42e6-879b-6629fefaa279', metadata={'chunk_id': 11.0, 'title': '엔트로피', 'url': 'https://ko.wikipedia.org/wiki/%EC%97%94%ED%8A%B8%EB%A1%9C%ED%94%BC', 'wiki_id': '158'}, page_content='엔트로피(, )는 열역학적 계의 유용하지 않은 (일로 변환할 수 없는) 에너지의 흐름을 설명할 때 이용되는 상태 함수다. 통계역학적으로, 주어진 거시적 상태에 대응하는 미시적 상태의 수의 로그로 생각할 수 있다. 엔트로피는 일반적으로 보존되지 않고, 열역학 제2법칙에 따라 시간에 따라 증가한다. 독일의 물리학자 루돌프 클라우지우스가 1850년대 초에 도입하였다. 대개 기호로 라틴 대문자 S를 쓴다.\n\n정의 \n엔트로피에는 열역학적 정의와 통계학적인 정의, 두 가지의 관련된 정의가 있다. 역사적으로, 고전 열역학적 정의가 먼저 발전하였다. 고전 열역학적인 관점에서, 그 이론은 원자나 분자 같은 수많은 성분들로 이루어져 있고, 학설의 안정성은 그러한 성분들의 평균적인 열특성으로 설명된다.\n이론을 구성하는 성분의 세부적인 성분들은 직접적으로 보이지 않는다. 그러나 그 특성은 온도, 압력, 엔트로피, 열용량과 같은 눈으로 볼 수 있는 평균적인 지표들에 의해 설명된다. 그 이론의 특성에 관한 고전적인 정의는 평형 상태임을 가정하였다. 엔트로피의 고전 열역학적 정의는 최근 비평형 열역학의 영역으로까지 확대되었다. 이후 엔트로피를 포함한 열특성은 눈으로 볼 수 없는 미시세계에서의 움직임에 대한 정역학적인 관점에서 새롭게 정의되었다. 그 예로 처음에는 기체로 여겨졌지만 시간이 지난 후에 광자, 음자, 스핀과 같은 양자 역학적으로 생각되는 뉴턴 입자가 있다. 이 이론의 특성에 대한 통계학적 설명은 고전적인 열역학을 사용하여 이론의 특성을 정의하는 것이 몇몇 변수의 영향을 받는 이론의 최종적인 상태를 예측하

In [34]:
question=[
    "고양시는 어떤곳 인가요?",
    "노량진은 어떤곳 인가요?",
    "딥러닝에 대해 어떻게 생각하나요?",
]
emb_question2=embedding.embed_documents(question)
emb_question2

[[0.0028094924055039883,
  -0.026035120710730553,
  0.022475939244031906,
  0.020194916054606438,
  -0.011021665297448635,
  -0.020686516538262367,
  -0.016704557463526726,
  -0.028670094907283783,
  0.0014723411295562983,
  -0.05883465334773064,
  0.0007005296065472066,
  -0.0004243119910825044,
  0.013735296204686165,
  -0.032347261905670166,
  0.007216683588922024,
  -0.026271088048815727,
  -0.04326077550649643,
  -0.004188429564237595,
  0.0326225571334362,
  0.05328940972685814,
  -0.039721257984638214,
  -0.000980126904323697,
  -0.05045779421925545,
  -0.01496429555118084,
  0.03311415761709213,
  -0.05179494619369507,
  -0.011090489104390144,
  0.027037983760237694,
  -0.00189265888184309,
  -0.017343638464808464,
  -0.012270328588783741,
  -0.02121744304895401,
  0.0022552136797457933,
  -0.06135164573788643,
  0.002792286453768611,
  -0.020037604495882988,
  0.025917137041687965,
  -0.0312657430768013,
  0.026546385139226913,
  0.01597699150443077,
  0.04636768624186516,
  0

In [40]:
for q in question:
    docs = vector_store.similarity_search(
        query=q,
        k=5,
        namespace="wiki-ns1"
    )
    print("질문:", q)
    for i, doc in enumerate(docs, start=1):
        print(f"결과 {i}: {doc.page_content[:100]}...")  # 청크 내용 일부 출력
        
    print("=" * 80)

질문: 고양시는 어떤곳 인가요?
결과 1: 소립자 물리학의 표준 모형(標準模型, )은 자연계의 기본 입자와, 중력을 제외한 그 상호작용 (강한 상호작용, 약한 상호작용, 전자기 상호작용)을 다루는 게이지 이론이다. 강력을 ...
결과 2: 선형대수학에서 벡터 공간(vector空間, , ) 또는 선형 공간(線型空間, )은 원소를 서로 더하거나 주어진 배수로 늘이거나 줄일 수 있는 공간이다. 체에 대한, 가군의 특수한 ...
결과 3: 양자역학(量子力學, )은 분자, 원자, 전자, 소립자 등 미시적인 계의 현상을 다루는 즉, 작은 크기를 갖는 계의 현상을 연구하는 물리학의 분야이다. 또는 아원자 입자 및 입자 집...
결과 4: 엔트로피(, )는 열역학적 계의 유용하지 않은 (일로 변환할 수 없는) 에너지의 흐름을 설명할 때 이용되는 상태 함수다. 통계역학적으로, 주어진 거시적 상태에 대응하는 미시적 상태...
결과 5: 6월 30일은 그레고리력으로 181번째(윤년일 경우 182번째) 날에 해당하며, 6월의 마지막 날이다.

사건 
 350년 - 게르만족 출신의 마그넨티우스가 네포티아누스의 봉기를 ...
질문: 노량진은 어떤곳 인가요?
결과 1: 대한민국 제16대 대통령 선거는 2002년 12월 19일 목요일 치뤄진 대통령 선거로, 21세기에 처음으로 치뤄진 대한민국 대통령 선거이다. 제15대 김대중 대통령의 차기 대통령을...
결과 2: 빅토르 로베르토비치 초이(, 1962년 6월 21일 ~ 1990년 8월 15일)는 소련의 록 가수이자, 싱어송라이터 겸 배우이며, 소련 록 음악 밴드 키노(КИНО)의 리더였다.
...
결과 3: 선형대수학에서 벡터 공간(vector空間, , ) 또는 선형 공간(線型空間, )은 원소를 서로 더하거나 주어진 배수로 늘이거나 줄일 수 있는 공간이다. 체에 대한, 가군의 특수한 ...
결과 4: 대수학의 기본 정리(代數學의 基本 定理 ; fundamental theorem of algebra)란 상수가 아닌 복소계수 다항식은 적어도 

## Augmentation

In [41]:
country="벨기에"
query=f"{country}의 여행을 위한 관광지와 문화"
docs=vector_store.similarity_search(query=query, k=5,namespace='wiki-ns1')

In [42]:
#augmentation context
context=""
for result in docs:
    context += result.page_content

In [43]:
#generation
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model='gpt-4o-mini')
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000002302965E660>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000230068C08C0>, root_client=<openai.OpenAI object at 0x00000230068C05C0>, root_async_client=<openai.AsyncOpenAI object at 0x000002302965F4A0>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********'))

In [44]:
from langchain.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ('system', '당신은 베테랑 여행 가이드입니다. 고객 맞춤 여행 일정 수립을 도와드립니다. 제한된 정보라도 최대한 활용해서 도움을 주세요'),
        ("""human', '저는 {country}로 여행을 계획 중입니다.
         아래 context에서 {country}관련 정보를 찾아 여행 조언을 해주세요.
         만약 직접적인 정보가 없다면, 수도나 관련 정보를 활용해 주세요.'
         context : {context}"""),
    ]
)

chat_template

ChatPromptTemplate(input_variables=['context', 'country'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 베테랑 여행 가이드입니다. 고객 맞춤 여행 일정 수립을 도와드립니다. 제한된 정보라도 최대한 활용해서 도움을 주세요'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'country'], input_types={}, partial_variables={}, template="human', '저는 {country}로 여행을 계획 중입니다.\n         아래 context에서 {country}관련 정보를 찾아 여행 조언을 해주세요.\n         만약 직접적인 정보가 없다면, 수도나 관련 정보를 활용해 주세요.'\n         context : {context}"), additional_kwargs={})])

In [45]:
prompt=chat_template.format_messages(country=country, context=context)
print(prompt)

[SystemMessage(content='당신은 베테랑 여행 가이드입니다. 고객 맞춤 여행 일정 수립을 도와드립니다. 제한된 정보라도 최대한 활용해서 도움을 주세요', additional_kwargs={}, response_metadata={}), HumanMessage(content='human\', \'저는 벨기에로 여행을 계획 중입니다.\n         아래 context에서 벨기에관련 정보를 찾아 여행 조언을 해주세요.\n         만약 직접적인 정보가 없다면, 수도나 관련 정보를 활용해 주세요.\'\n         context : 아쿠타가와 류노스케(, 1892년 3월 1일~1927년 7월 24일)는 일본의 소설가이다. 호는 징강당주인(澄江堂主人)이며 하이쿠 작가로서의 호는 가키(我鬼)이다.\n\n그의 작품은 대부분이 단편 소설이다. 「참마죽」, 「덤불 속」, 「지옥변」 등 주로 일본의 《곤자쿠 이야기집》·《우지슈이 이야기》 등 전통적인 고전들에서 제재를 취하였다. 또한 「거미줄(원제: 蜘蛛の糸)」, 「두자춘(杜子春)」 등 어린이를 위한 작품도 남겼으며, 예수를 학대한 유대인이 예수가 세상에 다시 올 때까지 방황한다는 상상력을 발휘한 「방황하는 유대인」도 있다.\n\n생애\n\n유년 시절 \n1892년(메이지 25년) 3월 1일 도쿄에서 우유 판매업자였던 아버지 니하라 도시조(新原敏三)와 어머니 후쿠(フク) 사이의 아들로 태어났다(아쿠타가와라는 성은 원래 그의 어머니쪽 성씨였다). 이때 태어난 시간이 공교롭게도 진년(辰年) 진월(辰月) 진일(辰日) 진시(辰時)였기 때문에 \'용(龍)\' 자를 이름에 넣어 류노스케(龍之介)라 짓게 되었다고 전하나, 실제 그가 태어난 1892년 3월 1일은 간지로는 임진년·임인월·임진일에 해당하며, 출생 시각에 대해서는 자료가 없기 때문에 확실한 것이 없다. 이름도 호적상으로는 \'龍之介\'이지만 그가 양자로 들어갔던 아쿠타가와 집안이나 졸업한 학교의 명단 등의 문서에는 \'龍之助\'로 되어 있다(아쿠타가와 

In [46]:
response=llm.invoke(prompt)
response.content

"벨기에의 여행을 계획하시는군요! 아쉽게도 제공해주신 정보는 일본의 작가 아쿠타가와 류노스케에 대한 내용입니다. 하지만 벨기에에 대한 정보를 제공할 수 있도록 몇 가지 유용한 조언을 드리겠습니다.\n\n### 벨기에 여행 정보\n\n1. **브뤼셀 (Brussels)**:\n   - 벨기에의 수도인 브뤼셀은 다양한 문화와 역사적 유산을 자랑합니다. 마네킨 피스(Manneken Pis)와 그랑 플라스(Grand Place)는 반드시 방문해 보세요.\n   - 브루셀의 유럽 연합 본부를 방문하면 유럽 정치에 대해서도 배울 수 있습니다.\n\n2. **브뤼жээ (Bruges)**:\n   - 중세 분위기를 자랑하는 이 도시는 '북유럽의 베네치아'로 알려져 있습니다. 운하를 따라서 보트를 타고, 미술관과 관람차를 즐길 수 있습니다.\n   - 벨기에 초콜릿을 현지에서 맛보는 것도 잊지 마세요.\n\n3. **겐트 (Ghent)**:\n   - 겐트는 아름다운 성과 대성당이 있는 도시로, 역사적인 건축물을 감상할 수 있습니다. 특히 성 미카엘 다리(Michael's Bridge)에서 보는 경치는 매우 아름답습니다.\n   - 벨기에 맥주 애호가라면 겐트의 여러 브루펍도 추천합니다.\n\n4. **암스텔담 (Antwerp)**: \n   - 패션과 다이아몬드로 유명한 암스텔담은 현대적인 감각과 전통이 잘 어우러진 도시입니다. 루벤스의 집(Rubenshuis)에서 안트워프의 유명한 화가의 작업을 볼 수 있습니다.\n   - 자전거를 빌려 도시를 돌아보는 것도 좋은 선택입니다.\n\n5. **벨기에 맥주와 초콜릿**:\n   - 벨기에는 세계적으로 유명한 맥주와 초콜릿이 있습니다. 다양한 종류의 맥주를 시음해 보고, 수제 초콜릿을 즐겨보세요.\n\n6. **교통**:\n   - 벨기에는 기차망이 잘 발달되어 있어서 브뤼셀에서 시작하여 다른 도시로 쉽게 이동할 수 있습니다. \n   - 자전거 대여 서비스 또한 잘 되어 있어 도심 구경에 유용합니다.\n\n여행 일정은 위 관

In [47]:
# 벡터 검색 + 키워드 필터링
def enhanced_search(country, vector_store):
    # 1단계: 벡터 검색
    vector_results = vector_store.similarity_search(
        query=f"{country} 여행 관광", 
        k=10, 
        namespace="wiki-ns1"
    )
    
    # 2단계: 국가명이 포함된 결과 우선순위
    country_related = []
    other_results = []
    
    for result in vector_results:
        if country.lower() in result.page_content.lower() or \
           "벨기에" in result.page_content or \
           "브뤼셀" in result.page_content:
            country_related.append(result)
        else:
            other_results.append(result)
    
    # 관련성 높은 결과 우선 반환
    final_results = (country_related + other_results)[:5]
    return final_results

results = enhanced_search("벨기에", vector_store)

In [ ]:
results